# Project 1 - Exploratory Data Analysis for Machine Learning
This notebook is a part of my first project required by IBM Machine Learning Program.

Data source: [Board Games - Tidy Tuesday](https://github.com/rfordatascience/tidytuesday/tree/master/data/2019/2019-03-12)

In this notebook, I will clean the data and explore the relationship between the target (game average rating) and potential predictors. I will also construct new features from the existing data when possible and perform hypothesis tests.

Notebook Contents:
> 1. Data Overview
> 2. Data Cleaning and Feature Engineering: Categorical Data
> 3. Data Cleaning and Feature Engineering: Numeric Data
> 4. Hypothesis Testing

## 1. Data Overview

In [ ]:
import pandas as pd
import seaborn as sns
sns.set()
%pylab inline
%config InlineBackend.figure_formats = ['retina']

In [ ]:
# Load in the dataset
data = pd.read_csv(r'C:\Users\Thanh Huynh\Documents\Projects\github\IBM-Machine-Learning\data\board_games.csv')

In [ ]:
# Create train set and test set
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2, random_state=0)

In [ ]:
# First look
train.head()

In [ ]:
# Examine columns
train.info()

In [ ]:
# Examine missing data
train.isnull().sum().sort_values()

There are missing data only in some of the categorical data. Let's clean those first.
## 2. Data Cleaning and Feature Engineering: Categorical Data

In [ ]:
# Have a look at all categorical variables
train.describe(include='object').T

In [ ]:
# Keep original data
df = train.copy()

In [ ]:
# Remove features that have too many missing values or those that can't help discriminate the target
df.drop(['game_id', 'description', 'image', 'name',
         'thumbnail', 'family', 'expansion', 'compilation'],
         axis=1, inplace=True)

### Counts derived from category aggregates

In [ ]:
# Select categorical variable names
category_cols = df.select_dtypes(np.object).columns

# Iterate through each column and count unique values
for cat in category_cols:
    num_unique_values = len(set(','.join(df[cat].dropna()).replace(', ', ',').split(',')))
    print(f'Number of unique values of {cat}:\t\t{num_unique_values}')

In [ ]:
# Count values in each categorical variable of each game
for cat in category_cols:
    df['num_' + cat] = [np.nan if x is np.nan 
                               else len(x) + 1 
                               for x in df[cat].str.findall(',')]

# Drop multi-level categorical variables
df.drop(['artist', 'designer', 'publisher'], axis=1, inplace=True)

In [ ]:
# Drop rows that have missing values
df.dropna(inplace=True)

# Current data shape
df.shape

Within a row, each categorical variable contains multiple values. I will transform these data to dummies.
### Categories derived from category aggregates

- Get a set of all unique values in each variable
- Create new columns based on these values
- Iterate through all rows and fill in dummy values for each new column
- Group these dummy variables if possible 

In [ ]:
# Create functions
def value_list(data: pd.DataFrame, col: str) -> list:
    """ Returns a list of unique values that included in a given column
    """
    # Join all values in the column then get a set of them
    value_set = set(','.join(df[col].dropna()).replace(', ', ',').split(','))

    # Make all string lower case for processing purpose
    repr_list = [x.lower() for x in list(value_set)]

    return repr_list

def create_dummies(data: pd.DataFrame, col: str, key_word: str) -> list:
    """ Creates dummies for a given category in a column
    """
    word_list = [1 if (x is not np.nan) and (key_word in str(x).lower())
                 else 0 if (x is not np.nan) and (key_word not in str(x).lower())
                 else np.nan for x in data[col]]
    return word_list


def dummy_loop(data: pd.DataFrame, col: str) -> pd.DataFrame:
    """ Returns a data frame of dummies 
    """
    # Iterate through columns
    for value in value_list(data, col):
        data[col + '_' + value] = create_dummies(data=data, col=col, key_word=value)
    
    # Drop the original columns
    data.drop(col, axis=1, inplace=True)
    return data

In [ ]:
# Print out game categories
value_list(df, 'category')

In [ ]:
# Print out game mechanics
value_list(df, 'mechanic')

In [ ]:
# Apply functions to get dummies
df = dummy_loop(dummy_loop(df, 'category'), 'mechanic')

# Print out first 5 rows
df.head()

In [ ]:
# Current data shape
df.shape

In [ ]:
# Check number of game categories
game_categories = df.filter(regex='^category_', axis=1)
game_categories.columns

In [ ]:
# Group categories based on BGG wiki
activity_categories = ['category_city building', 'category_civilization',
                       'category_educational', 'category_puzzle',
                       'category_racing', 'category_territory building',
                       'category_transportation']

component_categories = ['category_card game', 'category_collectible components',
                        'category_dice', 'category_electronic',
                        'category_miniatures']

nongame_categories = ['category_book', 'category_expansion for base-game',
                      'category_game system']

war_categories = list(df.filter(regex='war', axis=1).columns)
war_categories = war_categories + ['category_napoleonic', 'category_post-napoleonic',
                                   'category_pike and shot']

skills_categories = ['category_action / dexterity', 'category_bluffing',
                     'category_deduction', 'category_economic',
                     'category_math', 'category_number',
                     'category_memory', 'category_negotiation',
                     'category_real-time', 'category_spatial analysis',
                     'category_trivia', 'category_word game']

entertainment_categories = ['category_comic book / strip', 'category_movies / tv / radio theme',
                            'category_music', 'category_novel-based',
                            'category_video game theme']

nongroup_categories = ['category_party game', 'category_abstract strategy',
                       "category_children's game"]

In [ ]:
# Create functions
def dummy_group(data: pd.DataFrame, category: list) -> list:
    """
    Returns a list of dummies derived from input category
    """
    for i in category:
        if i not in list(data.columns):
            category.remove(i)
    dummies = [1 if x >= 1 else 0 for x in data[category].sum(axis=1)]
    data.drop(category, axis=1, inplace=True)
    return dummies

def plot_bar(data: pd.DataFrame,
             prefix: str, figsize: tuple,
             title: str):
    category_df = (df
                   .filter(regex=prefix, axis=1)
                   .sum()
                   .to_frame('count')
                   .sort_values('count')
    )
    ax = category_df.plot.barh(legend=None, figsize=figsize, linewidth=0)
    ax.set_yticklabels(list(category_df.sort_values('count').index.str.replace(prefix,'')), fontsize=7)
    ax.tick_params(axis='x', labelsize=7)
    ax.set_title(title)

In [ ]:
# Apply dummy function to each category
df['category_activity'] = dummy_group(df, activity_categories)
df['category_component'] = dummy_group(df, component_categories)
df['category_nongame'] = dummy_group(df, nongame_categories)
df['category_war'] = dummy_group(df, war_categories)
df['category_skills'] = dummy_group(df, skills_categories)
df['category_entertainment'] = dummy_group(df, entertainment_categories)

# Select games only
df = df[df['category_nongame'] == 0]
df.drop('category_nongame', axis=1, inplace=True)

In [ ]:
plot_bar(data=df,
         prefix='category_',
         figsize=(8,6),
         title='Number of Games by Category')

In [ ]:
plot_bar(data=df,
         prefix='mechanic_',
         figsize=(8,6),
         title='Number of Games by Mechanic')

In [ ]:
# Current data shape
df.shape

## 3. Data Cleaning and Engineering: Numeric Data

In [ ]:
# Select numeric data that are not binary
numeric_df = df.loc[:, ~df.isin([0,1]).all()]
numeric_df.shape

In [ ]:
# Some statistics
numeric_df.describe()

In [ ]:
# Select non zero rows only
df = df[(df.iloc[:,:6] > 0).all(axis=1)]

In [ ]:
# Create game age variable from year published
df['game_age'] = 2019 - df['year_published']

# Drop unuseful columns
df.drop(['max_playtime', 'min_playtime', 'users_rated'], axis=1, inplace=True)

In [ ]:
# Select data again
numeric_df = df.loc[:, ~df.isin([0,1]).all()]

# Current data shape
numeric_df.shape

In [ ]:
# Check statistics again
numeric_df.describe()

In [ ]:
# Create a functions to plot multiple bar charts
def hist_loop(data: pd.DataFrame,
              rows: int,
              cols: int,
              figsize: tuple):
    fig, axes = plt.subplots(rows,cols, figsize=figsize)
    for i, ax in enumerate(axes.flatten()):
        if i < len(data.columns):
            data[sorted(data.columns)[i]].plot.hist(bins=30, ax=ax)
            ax.set_title(f'{sorted(data.columns)[i]} distribution', fontsize=7)
            ax.tick_params(axis='x', labelsize=7)
            ax.tick_params(axis='y', labelsize=7)
            ax.get_yaxis().get_label().set_visible(False)
        else:
            fig.delaxes(ax=ax)
    fig.tight_layout()

In [ ]:
hist_loop(data=numeric_df,
          rows=3,
          cols=4,
          figsize=(12,6))

In [ ]:
# Create a function to check skewness
def skew_df(data: pd.DataFrame, skew_limit: float) -> pd.DataFrame:
    # Define a limit above which we will log transform
    skew_vals = data.skew()

    # Showing the skewed columns
    skew_cols = (skew_vals
                 .sort_values(ascending=False)
                 .to_frame('Skew')
                 .query('abs(Skew) > {}'.format(skew_limit))
    )
    return skew_cols

In [ ]:
# Print out skewed columns
skew_cols = skew_df(numeric_df, 0.75)
skew_cols

### Log transformation for skewed variables

In [ ]:
# Perform log transformation
for col in skew_cols.index.values:
    numeric_df['log_' + col] = numeric_df[col].apply(np.log1p)

In [ ]:
# Check skewness on log transformed data
log_df = numeric_df.filter(regex='^log_', axis=1)
skew_log_cols = skew_df(log_df, 0.75)
skew_log_cols

In [ ]:
# Plot log columns that have nearly normal distribution
log_df.drop(skew_log_cols.index, axis=1, inplace=True)
hist_loop(data=log_df,
          rows=3,
          cols=4,
          figsize=(12,6))

In [ ]:
# Join these new columns to our dataset
df = df.join(log_df).drop([x.replace('log_', '') for x in log_df.columns], axis=1)

In [ ]:
# Pairplot of numeric data
sns.pairplot(data=df, vars=['min_age', 'log_game_age', 'log_min_players', 'log_num_category',
                            'log_num_mechanic', 'log_playing_time', 'average_rating'],
             plot_kws=dict(alpha=.2, edgecolor='none'));

Observations from this plot:
- The target (average_rating) has a normal distribution.
- No strong linear relationship between the features and the target. Linear regression might not be well-suited to this problem.
- There might be a relationship between minimum age and playing time.
- I can try adding polynomial and interaction terms and examine their correlation with the target.

### Adding polynomial and interaction terms

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

# Instantiate and provide desired degree; 
# Note: degree=2 also includes intercept, degree 1 terms, and cross-terms
pf = PolynomialFeatures(degree=2)

In [ ]:
# Select features
feature_cols = ['min_age', 'log_game_age', 'log_min_players', 'log_num_category',
                'log_num_mechanic', 'log_playing_time']
features = df[feature_cols]

# fit and transform
pf.fit(features)
feat_array = pf.transform(features)

# Create a data frame
feat_df = pd.DataFrame(feat_array,
                       index=df.index,
                       columns=pf.get_feature_names(input_features=features.columns))

# Drop the intercept
feat_df.drop('1', axis=1, inplace=True)

# Add in the target
feat_df.insert(0, 'average_rating', df['average_rating'])

In [ ]:
# Plot a heatmap of correlations
with sns.axes_style('white'):
    fig, axes = plt.subplots(figsize=(8,8))
    mask = np.triu(np.ones_like(feat_df.corr(), dtype=bool))
    sns.heatmap(feat_df.corr(),
                mask=mask,
                cmap='binary',
                cbar=False,
                annot=True,
                annot_kws={'size':6},
                fmt='.2f')
    plt.title('Polynomial Features and Their Correlations')
    plt.tick_params(axis='x', labelsize=8)
    plt.tick_params(axis='y', labelsize=8)

This plot shows that polynomial and interaction terms do not have significantly higher correlations with the target comparing to the original features.

In [ ]:
# Join these new columns to our dataset
for col in feat_df.columns:
    if col in df.columns:
        feat_df.drop(col, axis=1, inplace=True)
df = df.join(feat_df)

### Binning numeric data

- Binning numeric data that cannot be scaled by log transformation
- Dummy transformation

In [ ]:
# Number of artists
pd.qcut(df['num_artist'], q=4, duplicates='drop')

In [ ]:
df['group_artist'] = pd.qcut(df['num_artist'],
                             q=4,
                             duplicates='drop',
                             labels=['two_or_less', 'three_or_more'])
df.drop('num_artist', axis=1, inplace=True)
df = pd.get_dummies(df, columns=['group_artist'], drop_first=True)

In [ ]:
# Number of designers
pd.qcut(df['num_designer'], q=4, duplicates='drop')

In [ ]:
df['group_designer'] = pd.qcut(df['num_designer'],
                               q=4,
                               duplicates='drop',
                               labels=['two_or_less', 'three_or_more'])
df.drop('num_designer', axis=1, inplace=True)
df = pd.get_dummies(df, columns=['group_designer'], drop_first=True)

In [ ]:
# Number of publishers
pd.qcut(df['num_publisher'], q=4, duplicates='drop')

In [ ]:
pd.qcut(df['num_publisher'], q=3, duplicates='drop')

In [ ]:
df['group_publisher'] = pd.qcut(df['num_publisher'],
                                q=3,
                                duplicates='drop',
                                labels=['three_or_less', 'four_or_more'])
df.drop('num_publisher', axis=1, inplace=True)
df = pd.get_dummies(df, columns=['group_publisher'], drop_first=True)

In [ ]:
# Number of maximum players
pd.qcut(df['max_players'], q=4, duplicates='drop')

In [ ]:
df['group_max_players'] = pd.qcut(df['max_players'],
                                  q=4,
                                  duplicates='drop',
                                  labels=['four_or_less', 'five_or_six', 'seven_or_more'])
df.drop('max_players', axis=1, inplace=True)
df = pd.get_dummies(df, columns=['group_max_players'], drop_first=True)

In [ ]:
# Year published
pd.qcut(df['year_published'], q=4, duplicates='drop')

In [ ]:
df['group_year_published'] = pd.qcut(df['year_published'],
                                     q=4,
                                     duplicates='drop',
                                     labels=['before_2001', 'between_2001_and_2009', 'between_2010_and_2013', 'between_2014_and_2016'])
df.drop('year_published', axis=1, inplace=True)
df = pd.get_dummies(df, columns=['group_year_published'], drop_first=True)

In [ ]:
# Final dataset
df.head()

In [ ]:
df.shape

## 4. Hypothesis Testing

In [ ]:
from scipy.stats import ttest_ind, t

In [ ]:
# Create a function to conduct Welch's test and plot distributions
def t_ind_test(s1: pd.Series,
               s2: pd.Series,
               equal_var=False) -> tuple:
    """
    Returns t-value and p-value
    """

    dfreedom = len(s1) + len(s2) - 2
    t_val, p_val = ttest_ind(s1, s2, equal_var=equal_var)
    return t_val, p_val

def test_loop(data: pd.DataFrame,
              prefix: str) -> pd.DataFrame:
    """
    Returns a table of t test result sorted by p-values 
    with colored bars (green for positive values and red for negative ones)
    """

    category_df = data.filter(regex=prefix, axis=1)
    category = sorted(list(category_df.columns))

    test_results = []
    for i in category:
        non_category_ratings = data[data[i] == 0]['average_rating']
        category_ratings = data[data[i] == 1]['average_rating']
        test_results.append((i.replace(prefix, ''),) + t_ind_test(category_ratings, non_category_ratings))

    test_results_df = (pd.DataFrame(test_results, columns=[prefix + 'name', 't-value', 'p-value'])
                       .sort_values('p-value')
                       .set_index(prefix + 'name')
    )
    test_results_df = test_results_df.style.bar(align='mid', color=['#d65f5f', '#5fba7d'])
    return test_results_df

In [ ]:
test_loop(df, 'category_')

In [ ]:
test_loop(df, 'mechanic_')

In [ ]:
test_loop(df, 'group_')